In [0]:
# Check mounts
# display(dbutils.fs.mounts())

In [0]:
# Check directory raw
# display(dbutils.fs.ls("/mnt/formula1dataplatformdl/raw"))

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

# Define schema
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)])

In [0]:
# Read circuits.csv with defined schema
circuits_df = spark.read.csv("/mnt/formula1dataplatformdl/raw/circuits.csv", header=True, schema=circuits_schema)
display(circuits_df)

In [0]:
# Check schema
# circuits_df.printSchema()

In [0]:
# Describe data
# display(circuits_df.describe())

In [0]:
# Select all columns except url (can be done with select as well, but then you have to specify all columns)
from pyspark.sql.functions import col

circuits_filtered = circuits_df.drop(col("url"))
display(circuits_filtered)

In [0]:
# Rename columns as needed

circuits_renamed = circuits_filtered.withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("circuitRef", "circuit_ref") \
    .withColumnRenamed("lat", "latitude") \
    .withColumnRenamed("lng", "longitude") \
    .withColumnRenamed("alt", "altitude")
display(circuits_renamed)

In [0]:
# Add ingestion date to df

from pyspark.sql.functions import current_timestamp

circuits_final = circuits_renamed.withColumn("ingestion_date", current_timestamp())
display(circuits_final)

In [0]:
# Write data to datalake as parquet

circuits_final.write.parquet("/mnt/formula1dataplatformdl/processed/circuits", mode="overwrite")

In [0]:
display(spark.read.parquet("/mnt/formula1dataplatformdl/processed/circuits"))